# Initialization

In [2]:
import os
from pathlib import Path
import signal
import time
import sys

# PyBoolNet initialization
from pyboolnet.repository import get_all_names, get_bnet
from pyboolnet.file_exchange import bnet2primes
from pyboolnet.trap_spaces import compute_trap_spaces, compute_steady_states

# trappist init
import trappist

# timeout with signals
class TimeOutException(Exception):
   pass

def handler(_signnum, _frame):
    print("Timeout")
    raise TimeOutException()
    raise AssertionError()
    raise KeyboardInterrupt()
    
signal.signal(signal.SIGALRM, handler)

<function __main__.handler(_signnum, _frame)>

In [3]:
# compute trap spaces using PyBoolNet
def bench_pyboolnet(models, max=1000, timeout=120, computation="min"):
    for model in models:
        print(model)
        primes = None
        signal.alarm(timeout)
        try:
            start = time.perf_counter()
            primes = bnet2primes(model + ".bnet")
            if computation == "fix":
                tspaces = compute_trap_spaces(primes, "min", max_output=max)
            else:
                tspaces = compute_steady_states(primes, max_output=max)
            end = time.perf_counter()
            print(f"{len(tspaces)} trap spaces {end - start:.2f}\n===")
        except:
            print(f"DNF{'(bnet2primes)' if primes is None else ''}\n===")
        signal.alarm(0)

In [4]:
# compute trap spaces using trappist
def bench_trappist(models, max=1000, timeout=120, method="asp", computation="min"):
    for model in models:
        print(model)
        signal.alarm(timeout)
        try:
            start = time.process_time()
            tspaces = trappist.compute_trap_spaces(model + ".bnet", max_output=max, method=method, computation=computation)
            print(f"{len(list(tspaces))} trap spaces {time.process_time() - start:.2f}\n===")
        except TimeOutException:
            print("DNF\n===")
        signal.alarm(0)

## Real models from the literature

In [5]:
# all models from the trappist's benchmark directory
models = [str(Path().absolute() / "models" / name[:-5]) for name in os.listdir("models") if name.endswith(".bnet")]

In [6]:
bench_trappist(models, max=0, timeout=120, method="asp", computation="min")

/home/giang-trinh/working/Trappist-min-max-fix/models/reduced-SIGNALING-IN-PROSTATE-CANCER_full
Compute min. trap spaces
24800 trap spaces 2.44
===
/home/giang-trinh/working/Trappist-min-max-fix/models/reduced-TLGLSurvival
Compute min. trap spaces
3236 trap spaces 0.15
===
/home/giang-trinh/working/Trappist-min-max-fix/models/reduced-IL_6_Signalling_full
Compute min. trap spaces
32768 trap spaces 0.89
===
/home/giang-trinh/working/Trappist-min-max-fix/models/reduced-CASCADE3
Compute min. trap spaces
58 trap spaces 0.05
===
/home/giang-trinh/working/Trappist-min-max-fix/models/reduced-A_model
Compute min. trap spaces
14 trap spaces 0.04
===


In [7]:
bench_trappist(models, max=0, timeout=120, method="asp", computation="max")

/home/giang-trinh/working/Trappist-min-max-fix/models/reduced-SIGNALING-IN-PROSTATE-CANCER_full
Compute max. trap spaces
35 trap spaces 0.11
===
/home/giang-trinh/working/Trappist-min-max-fix/models/reduced-TLGLSurvival
Compute max. trap spaces
27 trap spaces 0.02
===
/home/giang-trinh/working/Trappist-min-max-fix/models/reduced-IL_6_Signalling_full
Compute max. trap spaces
34 trap spaces 0.01
===
/home/giang-trinh/working/Trappist-min-max-fix/models/reduced-CASCADE3
Compute max. trap spaces
19 trap spaces 0.09
===
/home/giang-trinh/working/Trappist-min-max-fix/models/reduced-A_model
Compute max. trap spaces
13 trap spaces 0.02
===


In [8]:
bench_trappist(models, max=0, timeout=120, method="asp", computation="fix")

/home/giang-trinh/working/Trappist-min-max-fix/models/reduced-SIGNALING-IN-PROSTATE-CANCER_full
Compute fixed points
24800 trap spaces 2.37
===
/home/giang-trinh/working/Trappist-min-max-fix/models/reduced-TLGLSurvival
Compute fixed points
3236 trap spaces 0.11
===
/home/giang-trinh/working/Trappist-min-max-fix/models/reduced-IL_6_Signalling_full
Compute fixed points
32768 trap spaces 0.88
===
/home/giang-trinh/working/Trappist-min-max-fix/models/reduced-CASCADE3
Compute fixed points
58 trap spaces 0.05
===
/home/giang-trinh/working/Trappist-min-max-fix/models/reduced-A_model
Compute fixed points
14 trap spaces 0.02
===


In [6]:
bench_pyboolnet(models, max=100000, timeout=120, computation="min")

/home/giang-trinh/working/Trappist-min-max-fix/models/reduced-SIGNALING-IN-PROSTATE-CANCER_full
Timeout
DNF(bnet2primes)
===
/home/giang-trinh/working/Trappist-min-max-fix/models/reduced-TLGLSurvival
3236 trap spaces 0.49
===
/home/giang-trinh/working/Trappist-min-max-fix/models/reduced-IL_6_Signalling_full
32768 trap spaces 5.39
===
/home/giang-trinh/working/Trappist-min-max-fix/models/reduced-CASCADE3
58 trap spaces 0.66
===
/home/giang-trinh/working/Trappist-min-max-fix/models/reduced-A_model
14 trap spaces 0.14
===


In [7]:
bench_pyboolnet(models, max=100000, timeout=120, computation="max")

/home/giang-trinh/working/Trappist-min-max-fix/models/reduced-SIGNALING-IN-PROSTATE-CANCER_full
Timeout
DNF(bnet2primes)
===
/home/giang-trinh/working/Trappist-min-max-fix/models/reduced-TLGLSurvival
3236 trap spaces 0.47
===
/home/giang-trinh/working/Trappist-min-max-fix/models/reduced-IL_6_Signalling_full
32768 trap spaces 5.23
===
/home/giang-trinh/working/Trappist-min-max-fix/models/reduced-CASCADE3
58 trap spaces 0.61
===
/home/giang-trinh/working/Trappist-min-max-fix/models/reduced-A_model
14 trap spaces 0.13
===


In [8]:
bench_pyboolnet(models, max=100000, timeout=120, computation="fix")

/home/giang-trinh/working/Trappist-min-max-fix/models/reduced-SIGNALING-IN-PROSTATE-CANCER_full
Timeout
DNF(bnet2primes)
===
/home/giang-trinh/working/Trappist-min-max-fix/models/reduced-TLGLSurvival
3236 trap spaces 0.36
===
/home/giang-trinh/working/Trappist-min-max-fix/models/reduced-IL_6_Signalling_full
32768 trap spaces 4.95
===
/home/giang-trinh/working/Trappist-min-max-fix/models/reduced-CASCADE3
58 trap spaces 0.62
===
/home/giang-trinh/working/Trappist-min-max-fix/models/reduced-A_model
14 trap spaces 0.14
===
